In [22]:
!pip install -r requirements.txt

In [23]:
import os
import glob
import pandas as pd

In [24]:
PARQUET_PATH = "raw_data/*.parquet"
ZONE_LOOKUP = "raw_data/taxi_zone_lookup.csv"
OUTPUT_FILE = "cleaned_data/nyc_yellow_taxi_cleaned.csv"

In [25]:
files = glob.glob(PARQUET_PATH)
columns_needed = [
    'tpep_pickup_datetime',
    'tpep_dropoff_datetime',
    'passenger_count',
    'trip_distance',
    'PULocationID',
    'DOLocationID',
    'payment_type',
    'fare_amount',
    'tip_amount',
    'total_amount'
]

print("Reading parquet files...")
files = glob.glob(PARQUET_PATH)

df = pd.concat(
    [pd.read_parquet(f) for f in files],
    ignore_index=True
)

print(f"Rows after merge: {len(df)}")



Reading parquet files...
Rows after merge: 19760424


In [28]:
# Changing the column type to datetime
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])


# Removing invalid records
df = df[
    (df['trip_distance'] > 0) &
    (df['fare_amount'] > 0) &
    (df['total_amount'] > 0) &
    (df['passenger_count'] >= 1)
]

In [29]:
# Adding new Columns
df['trip_duration_min'] = (
    (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime'])
    .dt.total_seconds() / 60
)

df = df[
    (df['trip_duration_min'] >= 1) &
    (df['trip_duration_min'] <= 180)
]

df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour
df['pickup_day'] = df['tpep_pickup_datetime'].dt.day
df['pickup_month'] = df['tpep_pickup_datetime'].dt.month
df['pickup_weekday'] = df['tpep_pickup_datetime'].dt.day_name()

C:\Users\karan\AppData\Local\Temp\ipykernel_24564\3134756352.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['trip_duration_min'] = (


In [30]:
# Mapping payment type
payment_map = {
    1: 'Credit Card',
    2: 'Cash',
    3: 'No Charge',
    4: 'Dispute',
    5: 'Unknown'
}
df['payment_type_desc'] = df['payment_type'].map(payment_map)

In [31]:
# Joining Zone dataset to the taxi dataset
zones = pd.read_csv(ZONE_LOOKUP)

df = df.merge(
    zones[['LocationID', 'Borough', 'Zone']],
    left_on='PULocationID',
    right_on='LocationID',
    how='left'
)

df.rename(columns={
    'Borough': 'Pickup_Borough',
    'Zone': 'Pickup_Zone'
}, inplace=True)

df.drop(columns=['LocationID'], inplace=True)

In [32]:
final_cols = [
    'tpep_pickup_datetime',
    'tpep_dropoff_datetime',
    'pickup_hour',
    'pickup_day',
    'pickup_month',
    'pickup_weekday',
    'trip_duration_min',
    'passenger_count',
    'trip_distance',
    'fare_amount',
    'tip_amount',
    'total_amount',
    'payment_type_desc',
    'Pickup_Borough',
    'Pickup_Zone'
]

df = df[final_cols]

In [33]:
# Cleaned CSV Export

df.to_csv(OUTPUT_FILE, index=False)

print("Clean CSV created successfully!")
print(f"Final rows: {len(df)}")

Clean CSV created successfully!
Final rows: 14745943


# EAD

In [34]:
clean_df = pd.read_csv("cleaned_data/nyc_yellow_taxi_cleaned.csv")

In [35]:
print("Shape:", clean_df.shape)

Shape: (14745943, 15)


In [36]:
clean_df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,pickup_hour,pickup_day,pickup_month,pickup_weekday,trip_duration_min,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type_desc,Pickup_Borough,Pickup_Zone
0,2025-01-01 00:18:38,2025-01-01 00:26:59,0,1,1,Wednesday,8.350000,1.0,1.60,10.0,3.00,18.00,Credit Card,Manhattan,Sutton Place/Turtle Bay North
1,2025-01-01 00:32:40,2025-01-01 00:35:13,0,1,1,Wednesday,2.550000,1.0,0.50,5.1,2.02,12.12,Credit Card,Manhattan,Upper East Side North
2,2025-01-01 00:44:04,2025-01-01 00:46:01,0,1,1,Wednesday,1.950000,1.0,0.60,5.1,2.00,12.10,Credit Card,Manhattan,Lenox Hill West
3,2025-01-01 00:14:27,2025-01-01 00:20:01,0,1,1,Wednesday,5.566667,3.0,0.52,7.2,0.00,9.70,Cash,Manhattan,Washington Heights South
4,2025-01-01 00:21:34,2025-01-01 00:25:06,0,1,1,Wednesday,3.533333,3.0,0.66,5.8,0.00,8.30,Cash,Manhattan,Washington Heights South


In [37]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14745943 entries, 0 to 14745942
Data columns (total 15 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   tpep_pickup_datetime   object 
 1   tpep_dropoff_datetime  object 
 2   pickup_hour            int64  
 3   pickup_day             int64  
 4   pickup_month           int64  
 5   pickup_weekday         object 
 6   trip_duration_min      float64
 7   passenger_count        float64
 8   trip_distance          float64
 9   fare_amount            float64
 10  tip_amount             float64
 11  total_amount           float64
 12  payment_type_desc      object 
 13  Pickup_Borough         object 
 14  Pickup_Zone            object 
dtypes: float64(6), int64(3), object(6)
memory usage: 1.6+ GB


In [38]:
clean_df.describe

<bound method NDFrame.describe of          tpep_pickup_datetime tpep_dropoff_datetime  pickup_hour  pickup_day  \
0         2025-01-01 00:18:38   2025-01-01 00:26:59            0           1   
1         2025-01-01 00:32:40   2025-01-01 00:35:13            0           1   
2         2025-01-01 00:44:04   2025-01-01 00:46:01            0           1   
3         2025-01-01 00:14:27   2025-01-01 00:20:01            0           1   
4         2025-01-01 00:21:34   2025-01-01 00:25:06            0           1   
...                       ...                   ...          ...         ...   
14745938  2025-05-31 23:05:45   2025-05-31 23:26:55           23          31   
14745939  2025-05-31 23:32:52   2025-06-01 00:05:28           23          31   
14745940  2025-05-31 23:12:00   2025-05-31 23:20:03           23          31   
14745941  2025-05-31 23:26:32   2025-05-31 23:44:42           23          31   
14745942  2025-05-31 23:04:01   2025-05-31 23:13:24           23          31   

     

In [39]:
clean_df.isna().sum()

tpep_pickup_datetime         0
tpep_dropoff_datetime        0
pickup_hour                  0
pickup_day                   0
pickup_month                 0
pickup_weekday               0
trip_duration_min            0
passenger_count              0
trip_distance                0
fare_amount                  0
tip_amount                   0
total_amount                 0
payment_type_desc            0
Pickup_Borough            1896
Pickup_Zone              33495
dtype: int64